code from [here](https://blog.roboflow.com/how-to-use-segment-anything-model-sam/)

In [2]:
#Load the model
import torch
from segment_anything import sam_model_registry
!curl -O https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth #for Linux: wget {Link}
!pip install 'git+https://github.com/facebookresearch/segment-anything.git'
!pip install -q roboflow supervision 


DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
MODEL_TYPE = "vit_h"
CHECKPOINT_PATH = "sam_vit_h_4b8939.pth"
sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH)
sam.to(device=DEVICE)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2445M  100 2445M    0     0  11.9M      0  0:03:25  0:03:25 --:--:-- 12.1M0     0  11.8M      0  0:03:27  0:00:18  0:03:09 12.1MM    0     0  11.8M      0  0:03:26  0:00:21  0:03:05 12.1M    0  11.9M      0  0:03:25  0:00:31  0:02:54 12.0M5  0:00:43  0:02:42 12.0M     0  11.9M      0  0:03:24  0:01:08  0:02:16 12.1M    0     0  11.9M      0  0:03:25  0:01:22  0:02:03 12.1M39M    0     0  11.9M      0  0:03:25  0:01:27  0:01:58 11.6M1.9M      0  0:03:25  0:01:34  0:01:51 12.0M     0  0:03:25  0:01:50  0:01:35 12.1M646M    0     0  11.9M      0  0:03:25  0:02:18  0:01:07 12.1M   0  0:03:25  0:02:36  0:00:49 12.1M0  11.9M      0  0:03:25  0:03:09  0:00:16 11.9M1.9M      0  0:03:25  0:03:12  0:00:13 12.1M
  Cloning https://github.com/facebookresearch/segment-anything.git to /private/var/folders/tt/plrj7c_x5c769tsqdqgf30w40000gn/T/

Sam(
  (image_encoder): ImageEncoderViT(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 1280, kernel_size=(16, 16), stride=(16, 16))
    )
    (blocks): ModuleList(
      (0-31): 32 x Block(
        (norm1): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=1280, out_features=3840, bias=True)
          (proj): Linear(in_features=1280, out_features=1280, bias=True)
        )
        (norm2): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (lin1): Linear(in_features=1280, out_features=5120, bias=True)
          (lin2): Linear(in_features=5120, out_features=1280, bias=True)
          (act): GELU(approximate='none')
        )
      )
    )
    (neck): Sequential(
      (0): Conv2d(1280, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): LayerNorm2d()
      (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (3): LayerNorm2d

In [10]:
# Define image path here
IMAGE_PATH = '/Users/lisa/Documents/Master/sam-lab/test_image.jpeg'


In [11]:
# Automated Mask (Instance Segmentation) Generation with SAM
import cv2
from segment_anything import SamAutomaticMaskGenerator

mask_generator = SamAutomaticMaskGenerator(sam)

image_bgr = cv2.imread(IMAGE_PATH)
image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
result = mask_generator.generate(image_rgb)


error: OpenCV(4.8.0) /Users/xperience/GHA-OpenCV-Python/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [12]:
cv2.imshow("Image", image_bgr)

error: OpenCV(4.8.0) /Users/xperience/GHA-OpenCV-Python/_work/opencv-python/opencv-python/opencv/modules/highgui/src/window.cpp:971: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'imshow'


In [ ]:
#Annotate Masks with Supervision
import supervision as sv

mask_annotator = sv.MaskAnnotator(color_map = "index")
detections = sv.Detections.from_sam(result)
annotated_image = mask_annotator.annotate(image_bgr, detections)

In [ ]:
#Generate Segmentation Mask with Bounding Box

import cv2
from segment_anything import SamPredictor

mask_predictor = SamPredictor(sam)

image_bgr = cv2.imread(IMAGE_PATH)
image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
mask_predictor.set_image(image_rgb)

box = np.array([70, 247, 626, 926])
masks, scores, logits = mask_predictor.predict(
    box=box,
    multimask_output=True
)